<a href="https://colab.research.google.com/github/Hemanthkumar2112/huggingface_onnx/blob/main/huggingface_onnx_quantized_model_for_sequence_classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install git+https://github.com/huggingface/optimum.git#egg=optimum[onnxruntime]

In [3]:
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
save_directory = "tmp/onnx/"

# Load a model from transformers and export it to ONNX
ort_model = ORTModelForSequenceClassification.from_pretrained(model_checkpoint, from_transformers=True)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Save the onnx model and tokenizer
ort_model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('tmp/onnx/tokenizer_config.json',
 'tmp/onnx/special_tokens_map.json',
 'tmp/onnx/vocab.txt',
 'tmp/onnx/added_tokens.json',
 'tmp/onnx/tokenizer.json')

In [4]:
from optimum.onnxruntime.configuration import AutoQuantizationConfig
from optimum.onnxruntime import ORTQuantizer

# Define the quantization methodology
qconfig = AutoQuantizationConfig.arm64(is_static=False, per_channel=False)
quantizer = ORTQuantizer.from_pretrained(ort_model)

# Apply dynamic quantization on the model
quantizer.quantize(save_dir=save_directory, quantization_config=qconfig)

PosixPath('tmp/onnx')

In [5]:
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import pipeline, AutoTokenizer

model = ORTModelForSequenceClassification.from_pretrained(save_directory, file_name="model_quantized.onnx")
tokenizer = AutoTokenizer.from_pretrained(save_directory)

cls_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)



In [9]:
%%time
cls_pipeline("I love burritos!")

CPU times: user 20.1 ms, sys: 0 ns, total: 20.1 ms
Wall time: 22.8 ms


[{'label': 'POSITIVE', 'score': 0.9996811151504517}]